# Apache Kafka PySpark streaming

Welcome to the Kafka Streaming lab class! In this class, we will be learning how to use Apache Kafka to process and analyze real-time data streams. Kafka is a distributed streaming platform that is used for building real-time data pipelines and streaming applications. It is highly scalable and fault-tolerant, making it an ideal choice for handling large volumes of data. Let's get started!


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.jars.packages", "spark-sql-kafka-0-10_2.12").getOrCreate()

## Exercise 1
Connnect to the Kafka at port 9092 and subscribe to topic *page_view_events*

In [4]:
df = spark.readStream.format("kafka") \
      .option("kafka.bootstrap.servers", "kafka:9092") \
      .option("subscribe", "page_view_events") \
      .load()

df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



## Exercise 2
By default the spark-sql-kafka returns the `key` and `value` fields as a binary.
1. Cast both of them to `string`
4. Create a query named `q1` with the outputMode set to `append` and format set to `memory`
6. Define `StructType` that contains following fields: `ts: long`, `state: string`, `city: string`
8. Create a DataFrame with the `ts`, `state` and `city` fields that are encoded in JSON inside `value` field.

In [5]:
df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [6]:
query = df.writeStream.outputMode("append").format("memory").queryName("q1").start()

In [7]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

schema = StructType(
    [
        StructField("ts", LongType(), True),
        StructField("state", StringType(), True),
        StructField("city", StringType(), True)
    ]
)

In [8]:
from pyspark.sql.functions import col, from_json

df = spark.sql("SELECT value FROM q1") \
    .withColumn("value", from_json("value", schema)) \
    .select(col("value.*"))

## Exercise 3

Print the count of the events in each state sorted by the count every 10 seconds. Restart the kafka instance and find a place where the pyspark has reconnected with the kafka.

In [9]:
from IPython.display import display, clear_output
from time import sleep

while True:
    # clear_output(wait=True)
    display(query.status)
    display(df.groupBy("state").count().sort(col("count").desc()).show())
    display(df.count())
    sleep(10)

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

+-----+-----+
|state|count|
+-----+-----+
|   TX|    2|
|   KY|    1|
|   LA|    1|
|   TN|    1|
|   CA|    1|
|   MN|    1|
|   NJ|    1|
|   IL|    1|
|   MO|    1|
|   NY|    1|
+-----+-----+



None

11

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

...

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

+-----+-----+
|state|count|
+-----+-----+
|   TX|    6|
|   VA|    5|
|   CA|    5|
|   OH|    4|
|   FL|    4|
|   IL|    3|
|   NY|    3|
|   PA|    3|
|   SC|    2|
|   WA|    2|
|   MI|    2|
|   NC|    2|
|   NE|    2|
|   MN|    2|
|   TN|    2|
|   LA|    1|
|   NJ|    1|
|   KY|    1|
|   IN|    1|
|   MO|    1|
+-----+-----+
only showing top 20 rows



None

62

KeyboardInterrupt: 

# Exercise 4

Create a expression that groups the events into 10 seconds time windows. Print the last few rows, restart the kafka and show that no data was lost.

In [ ]:
from pyspark.sql.functions import window
import pandas as pd

while True:
    #clear_output(wait=True)
    #display(query.status)
    display(pd.DataFrame(df
        .selectExpr("from_unixtime(CAST(ts/1000 as BIGINT)) as ts")
        .groupBy(window("ts", "10 seconds"))
        .count()
        .sort("window")
        .tail(15)
    ))
    sleep(10)

,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


...

,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5


,0,1
0,"(2023-01-17 08:38:20, 2023-01-17 08:38:30)",5
1,"(2023-01-17 08:38:30, 2023-01-17 08:38:40)",7
2,"(2023-01-17 08:38:40, 2023-01-17 08:38:50)",4
3,"(2023-01-17 08:38:50, 2023-01-17 08:39:00)",7
4,"(2023-01-17 08:39:00, 2023-01-17 08:39:10)",12
5,"(2023-01-17 08:39:10, 2023-01-17 08:39:20)",4
6,"(2023-01-17 08:39:20, 2023-01-17 08:39:30)",6
7,"(2023-01-17 08:39:30, 2023-01-17 08:39:40)",12
8,"(2023-01-17 08:39:40, 2023-01-17 08:39:50)",4
9,"(2023-01-17 08:39:50, 2023-01-17 08:40:00)",5
